# Assignment 4.1: Model Store

**Student name: John Kalaiselvan**

## Part 1: Set Up Model Group

### Set up the SageMaker AI Boto3 client.

In [5]:
import time
import os
from sagemaker import get_execution_role, session
import boto3


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [6]:
region = boto3.Session().region_name

role = get_execution_role()

sm_client = boto3.client('sagemaker', region_name=region)

### Create the Model Group

In [11]:
import time
model_package_group_name = "xgboost-breast-cancer-detection"
model_package_group_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageGroupDescription" : ( 
        "XGBoost models for breast cancer detection using clinical feature data."
        "Tracks versions across feature engineering, hyperparameter tuning," 
        "and training data updates."
    )
}

create_model_package_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
print('ModelPackageGroup Arn : {}'.format(create_model_package_group_response['ModelPackageGroupArn']))

ModelPackageGroup Arn : arn:aws:sagemaker:us-east-1:823460696669:model-package-group/xgboost-breast-cancer-detection


### Describe Model Package Group

In [12]:
describe_response = sm_client.describe_model_package_group(
    ModelPackageGroupName="xgboost-breast-cancer-detection"
)

In [13]:
describe_response

{'ModelPackageGroupName': 'xgboost-breast-cancer-detection',
 'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:823460696669:model-package-group/xgboost-breast-cancer-detection',
 'ModelPackageGroupDescription': 'XGBoost models for breast cancer detection using clinical feature data.Tracks versions across feature engineering, hyperparameter tuning,and training data updates.',
 'CreationTime': datetime.datetime(2026, 2, 1, 10, 35, 23, 458000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:823460696669:user-profile/d-24shzxj3uurr/jkalaiselvan',
  'UserProfileName': 'jkalaiselvan',
  'DomainId': 'd-24shzxj3uurr',
  'IamIdentity': {'Arn': 'arn:aws:sts::823460696669:assumed-role/LabRole/SageMaker',
   'PrincipalId': 'AROA37OQVCJORZKK6N5DG:SageMaker'}},
 'ModelPackageGroupStatus': 'Completed',
 'ResponseMetadata': {'RequestId': '973be2a6-ea08-490d-a357-ead2bd1ed18f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '973be2a6-ea08-490d-a357-e

## Part 2: Set Up Model Package

### Find the Trained Model Artifacts

In [1]:
import boto3

sm = boto3.client("sagemaker")

jobs = sm.list_training_jobs(
    SortBy="CreationTime",
    SortOrder="Descending",
    MaxResults=5
)

[j["TrainingJobName"] for j in jobs["TrainingJobSummaries"]]


['xgb-2026-02-01-10-14-48']

In [2]:
training_job_name = "xgb-2026-02-01-10-14-48"

job_desc = sm.describe_training_job(
    TrainingJobName=training_job_name
)

model_data_url = job_desc["ModelArtifacts"]["S3ModelArtifacts"]


In [3]:
model_data_url

's3://sagemaker-us-east-1-823460696669/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2026-02-01-10-14-48/xgb-2026-02-01-10-14-48/output/model.tar.gz'

### Create Model Package

In [18]:
#xgboost_image_uri = f"{region}.dkr.ecr.amazonaws.com/sagemaker-xgboost:1.7-1"
xgboost_image_uri = f"683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1"
xgboost_image_uri

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1'

In [19]:
import boto3

sm_client = boto3.client("sagemaker")

create_response = sm_client.create_model_package(
    ModelPackageGroupName="xgboost-breast-cancer-detection",
    ModelPackageDescription=(
        "XGBoost classification model for breast cancer detection. "
        "Trained on clinical diagnostic features. "
        "This version represents the baseline production candidate."
    ),
    InferenceSpecification={
        "Containers": [
            {
                "Image": xgboost_image_uri,
                "ModelDataUrl": model_data_url,
                "Environment": {
                    "SAGEMAKER_PROGRAM": "inference.py",
                    "SAGEMAKER_SUBMIT_DIRECTORY": "/opt/ml/model"
                }
            }
        ],
        "SupportedContentTypes": ["text/csv"],
        "SupportedResponseMIMETypes": ["text/csv"]
    },
    ModelApprovalStatus="PendingManualApproval"
)


In [20]:
create_response

{'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:823460696669:model-package/xgboost-breast-cancer-detection/3',
 'ResponseMetadata': {'RequestId': 'fd25c4b9-1cb2-4a85-9961-573c60795e23',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fd25c4b9-1cb2-4a85-9961-573c60795e23',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'content-security-policy': "frame-ancestors 'none'",
   'cache-control': 'no-cache, no-store, must-revalidate',
   'x-content-type-options': 'nosniff',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '110',
   'date': 'Sun, 01 Feb 2026 11:25:04 GMT'},
  'RetryAttempts': 0}}

### Describe the Model Package

In [21]:
model_package_arn = create_response["ModelPackageArn"]

describe_response = sm_client.describe_model_package(
    ModelPackageName=model_package_arn
)


In [22]:
describe_response

{'ModelPackageGroupName': 'xgboost-breast-cancer-detection',
 'ModelPackageVersion': 3,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:823460696669:model-package/xgboost-breast-cancer-detection/3',
 'ModelPackageDescription': 'XGBoost classification model for breast cancer detection. Trained on clinical diagnostic features. This version represents the baseline production candidate.',
 'CreationTime': datetime.datetime(2026, 2, 1, 11, 25, 4, 8000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1',
    'ImageDigest': 'sha256:b4f13edb198529c460692015797fa1ca6a8ff1ed64a149297174d922121b8fc4',
    'ModelDataUrl': 's3://sagemaker-us-east-1-823460696669/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2026-02-01-10-14-48/xgb-2026-02-01-10-14-48/output/model.tar.gz',
    'Environment': {'SAGEMAKER_PROGRAM': 'inference.py',
     'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model'},
    'ModelDataE

## Part 3: Write the Model Card